# Storypoint Prediction: Regression Approach

## Preparation


In [3]:
import os
from IPython.display import clear_output

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from scipy.sparse import csr_matrix, hstack, vstack

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, f1_score, precision_score, recall_score, accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_val_score, learning_curve, validation_curve
from trainer import GridSearchCVTrainer

project_name = 'moodle'

### Plot learning curve

In [4]:
def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,
                        n_jobs=-1, train_sizes=np.linspace(.1, 1.0, 10)):
    plt.figure()        # Create new figure
    plt.title(title)    # Set title of the figure
    
    # Set y-axis limits: ylim=(min, max)
    if ylim is not None:
        plt.ylim(*ylim)  
        
    plt.xlabel("Training examples")  # Set x-axis label
    plt.ylabel("Score")               # Set y-axis label
    
    # Generate learning curve data
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes, scoring='neg_mean_squared_error')
    
    train_scores_mean = np.mean(train_scores, axis=1)  # Calculate mean of training scores
    train_scores_std = np.std(train_scores, axis=1)    # Calculate standard deviation of training scores
    test_scores_mean = np.mean(test_scores, axis=1)    # Calculate mean of test scores
    test_scores_std = np.std(test_scores, axis=1)      # Calculate standard deviation of test scores
    
    plt.grid()  # Display grid
    
    # Fill the area between the mean training score and the mean +/- std training score
    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    
    # Fill the area between the mean test score and the mean +/- std test score
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                    test_scores_mean + test_scores_std, alpha=0.1, color="g")
    
    # Plot mean training score as points
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    # Plot mean test score as points
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Validation score")
    
    plt.legend(loc="best")  # Display legend
    return plt

### Plot validation curve

In [5]:
def plot_validation_curve(estimator, title, X, y, param_name, param_range,
                          y_lim=None, cv=10, n_jobs=-1):
    train_scores, val_scores = validation_curve(estimator=estimator, X=X, y=y, 
                                                 param_name=param_name, param_range=param_range, 
                                                 cv=cv, n_jobs=n_jobs,
                                                 scoring='neg_mean_squared_error')
    
    # Calculate mean and standard deviation of training and validation scores
    train_mean = np.mean(train_scores, axis=1)
    tran_std = np.std(train_scores, axis=1)
    val_mean = np.mean(val_scores, axis=1)
    val_std = np.std(val_scores, axis=1)
    print(val_mean)
    
    # Plot train scores
    plt.plot(param_range, train_mean, color='r', marker='o', markersize=5, label='Training score')
    plt.fill_between(param_range, train_mean + tran_std, train_mean - tran_std, alpha=0.15, color='r')
    
    # Plot validation scores
    plt.plot(param_range, val_mean, color='g', linestyle='--', marker='s', markersize=5, label='Validation score')
    plt.fill_between(param_range, val_mean + val_std, val_mean - val_std, alpha=0.15, color='g')
    
    plt.title(title)        # Set title of the plot
    plt.grid()              # Display grid
    plt.xscale('log')       # Set x-axis scale to log
    plt.legend(loc='best')  # Display legend
    plt.xlabel('Parameter') # Set x-axis label
    plt.ylabel('Score')     # Set y-axis label
    
    # Set y-axis limits
    if y_lim != None:
        plt.ylim(y_lim)
    
    return plt

### Evaluate model

In [11]:
def evaluate_model(model, model_name, X_test, y_test, y_logscale=False, save_to_file=True):
    y_pred = model.predict(X_test)
    if(y_logscale):
        y_pred = np.exp(y_pred)
        
    lines = [model_name + '\'s evaluation results:']
    
    mse = mean_squared_error(y_test, y_pred) 
    rmse = np.sqrt(mse) 
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    lines.append(f' - Mean squared error:      {mse:.2f}')
    lines.append(f' - Root mean squared error: {rmse:.2f}')
    lines.append(f' - Mean absolute error:     {mae:.2f}')
    lines.append(f' - R2 error:                {r2:.2f}')
    
    y_pred = np.round(y_pred).astype(int)
    f1 = f1_score(y_test, y_pred, average='weighted')
    precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
    recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)
    accuracy = accuracy_score(y_test, y_pred)

    lines.append(f' - F1 score:                {f1:.2f}')
    lines.append(f' - Precision:               {precision:.2f}')
    lines.append(f' - Recall:                  {recall:.2f}')
    lines.append(f' - Accuracy:                {accuracy:.2f}')
    lines.append('-------------------------------------------------')
    lines.append('')
    
    # Save to file
    
    filename = 'result/' + project_name + '.txt'
    directory = os.path.dirname(filename)
    if not os.path.exists(directory):
        os.makedirs(directory)
        
    with open(filename, 'a') as f:
        for line in lines:
            print(line)
            if(save_to_file):
                f.write(line + '\n')

## Dataset set-up

### Bag of Words preprocessing

This is a Bag of Words preprocess approach. I will use 2 CountVectorizer from sklearn to change title and description to two 2 vectors and then concatenate them together. In the rest of this notebook, I will use cross-validation instead hold-out. Therefore, I will join the validation set with training set.

In [7]:
# Import and remove NaN value


data_train = pd.concat([pd.read_csv(project_name + '/' + project_name + '_dataset_train.csv'),
                       pd.read_csv(project_name + '/' + project_name + '_dataset_valid.csv')])
data_test = pd.read_csv(project_name + '/' + project_name + '_dataset_test.csv')

data_train['description'].replace(np.nan, '', inplace=True)
data_test['description'].replace(np.nan, '', inplace=True)

# Vectorize title
title_vectorizer = CountVectorizer(ngram_range=(1, 2), min_df=2)
title_vectorizer.fit(pd.concat([data_train['title'], data_test['title']]))

# Vectorize description
description_vectorizer = CountVectorizer(ngram_range=(1, 2), min_df=2)
description_vectorizer.fit(pd.concat([data_train['description'], data_test['description']]))


X_train = hstack([title_vectorizer.transform(data_train['title']).astype(float),
                  description_vectorizer.transform(data_train['description']).astype(float),
                  data_train['title'].apply(lambda x : len(x)).to_numpy().reshape(-1, 1),
                  data_train['description'].apply(lambda x : len(x)).to_numpy().reshape(-1, 1)])

y_train = data_train['storypoint'].to_numpy().astype(float)

X_test = hstack([title_vectorizer.transform(data_test['title']).astype(float),
                  description_vectorizer.transform(data_test['description']).astype(float),
                  data_test['title'].apply(lambda x : len(x)).to_numpy().reshape(-1, 1),
                  data_test['description'].apply(lambda x : len(x)).to_numpy().reshape(-1, 1)])

y_test = data_test['storypoint'].to_numpy().astype(float)

In [8]:
print('Check training dataset\'shape:', X_train.shape, y_train.shape)
print('Check testing dataset\'shape:', X_test.shape, y_test.shape)

Check training dataset'shape: (1050, 6776) (1050,)
Check testing dataset'shape: (116, 6776) (116,)


I will use log-scale the label to get a normal distribution of it.

In [9]:
y_train_log = np.log(y_train)

## Model training

### Linear Regressor

In [15]:
from sklearn.linear_model import ElasticNet

Define params-grid:

In [9]:
dict_param = {
    'alpha': [.0001, .001, .01, .1, 1, 10, 100, 1000, 10000],
    'l1_ratio': [.0, .2, .4, .6, .8, .1],
    'max_iter': [10**5]
}

In [10]:
gridsearch = GridSearchCVTrainer(name='Elastic Net', model=ElasticNet(), param_grid=dict_param, cv=5, n_jobs=5)
gridsearch.fit(X_train, y_train_log)

elastic_model = gridsearch.best_estimator_
elastic_model.fit(X_train, y_train_log)

Training combination 54/54


c:\Users\aupho\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:658: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 355.71805629733757, tolerance: 0.11916336274031106
  model = cd_fast.sparse_enet_coordinate_descent(


ElasticNet(alpha=0.1, l1_ratio=0.0, max_iter=100000)

In [11]:
evaluate_model(elastic_model, 'Elastic Net model', X_test, y_test, y_logscale=True)

Elastic Net model's evaluation results:
 - Mean squared error:      108.18
 - Root mean squared error: 10.40
 - Mean absolute error:     7.39
 - R2 error:                -27.19
 - F1 score:                0.02
 - Precision:               0.44
 - Recall:                  0.02
 - Accuracy:                0.02
-------------------------------------------------



In [12]:
elastic_model.get_params()

{'alpha': 0.1,
 'copy_X': True,
 'fit_intercept': True,
 'l1_ratio': 0.0,
 'max_iter': 100000,
 'positive': False,
 'precompute': False,
 'random_state': None,
 'selection': 'cyclic',
 'tol': 0.0001,
 'warm_start': False}

### Support Vector Regressor

In [16]:
from sklearn.svm import SVR

In [14]:
dict_param = {
    'C': [.0001, .001, .01, .1, 1, 10, 100, 1000, 10000],
    'gamma': np.logspace(-9, 3, 13),
    'kernel': ['rbf']
}

In [15]:
grid_search = GridSearchCVTrainer(name="Support Vector Regressor",model=SVR(), param_grid=dict_param, cv=5, n_jobs=5)
grid_search.fit(X_train, y_train_log)

svr_rbf_model = grid_search.best_estimator_
svr_rbf_model.fit(X_train, y_train_log)

Training combination 117/117


SVR(C=1000, gamma=1e-05)

In [16]:
evaluate_model(svr_rbf_model, 'SVR RBF model', X_test, y_test, y_logscale=True)

SVR RBF model's evaluation results:
 - Mean squared error:      92.61
 - Root mean squared error: 9.62
 - Mean absolute error:     7.12
 - R2 error:                -23.13
 - F1 score:                0.01
 - Precision:               0.01
 - Recall:                  0.03
 - Accuracy:                0.03
-------------------------------------------------



In [17]:
svr_rbf_model.get_params()

{'C': 1000,
 'cache_size': 200,
 'coef0': 0.0,
 'degree': 3,
 'epsilon': 0.1,
 'gamma': 1e-05,
 'kernel': 'rbf',
 'max_iter': -1,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

### Random Forest Regressor

In [17]:
from sklearn.ensemble import RandomForestRegressor

In [19]:
dict_param = {
    'max_depth' : [1000, 2000, 5000],
    'min_samples_split': [25, 200, 1000],
    'min_samples_leaf': [1, 2, 3, 4],
    'max_features': [50, 100, 200],
    'n_estimators': [1024]
}

In [20]:
grid_search = GridSearchCVTrainer(name="Random Forest Regressor", 
                                  model=RandomForestRegressor(), 
                                  param_grid=dict_param, cv = 5, n_jobs=5)
grid_search.fit(X_train, y_train_log)

rfr_model = grid_search.best_estimator_
rfr_model.fit(X_train, y_train_log)

Training combination 108/108


RandomForestRegressor(max_depth=2000, max_features=50, min_samples_split=25,
                      n_estimators=1024)

In [21]:
evaluate_model(rfr_model, 'Random Forest model', X_test, y_test, y_logscale=True)

Random Forest model's evaluation results:
 - Mean squared error:      42.46
 - Root mean squared error: 6.52
 - Mean absolute error:     6.02
 - R2 error:                -10.06
 - F1 score:                0.01
 - Precision:               0.00
 - Recall:                  0.01
 - Accuracy:                0.01
-------------------------------------------------



In [22]:
rfr_model.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'squared_error',
 'max_depth': 2000,
 'max_features': 50,
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 25,
 'min_weight_fraction_leaf': 0.0,
 'monotonic_cst': None,
 'n_estimators': 1024,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

### XGBoost

In [18]:
from xgboost import XGBRegressor

In [24]:
dict_param = {
    'eta' : np.linspace(0.01, 0.2, 3),
    'gamma': np.logspace(-2, 2, 5),
    'max_depth': np.asarray([3, 5, 7, 9]).tolist(),
    'min_child_weight': np.logspace(-2, 2, 5),
    'subsample': np.asarray([0.5, .1]),
    'reg_alpha': np.asarray([0.0, 0.05]),
    'n_estimators': np.asarray([10, 20, 50, 100]).tolist(),
}

In [25]:
grid_search = GridSearchCVTrainer(name='XGBoost Regressor',model=XGBRegressor(), param_grid=dict_param, cv = 5, n_jobs=5)
grid_search.fit(X_train, y_train_log)

xgb_model = grid_search.best_estimator_
xgb_model.fit(X_train, y_train_log)

Training combination 4800/4800


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eta=0.105, eval_metric=None,
             feature_types=None, gamma=1.0, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=9,
             max_leaves=None, min_child_weight=0.01, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=50,
             n_jobs=None, num_parallel_tree=None, ...)

In [26]:
evaluate_model(xgb_model, 'XGBoost Regressor model', X_test, y_test, y_logscale=True)

XGBoost Regressor model's evaluation results:
 - Mean squared error:      62.85
 - Root mean squared error: 7.93
 - Mean absolute error:     6.84
 - R2 error:                -15.38
 - F1 score:                0.00
 - Precision:               0.00
 - Recall:                  0.00
 - Accuracy:                0.00
-------------------------------------------------



In [27]:
xgb_model.get_params()

{'objective': 'reg:squarederror',
 'base_score': None,
 'booster': None,
 'callbacks': None,
 'colsample_bylevel': None,
 'colsample_bynode': None,
 'colsample_bytree': None,
 'device': None,
 'early_stopping_rounds': None,
 'enable_categorical': False,
 'eval_metric': None,
 'feature_types': None,
 'gamma': 1.0,
 'grow_policy': None,
 'importance_type': None,
 'interaction_constraints': None,
 'learning_rate': None,
 'max_bin': None,
 'max_cat_threshold': None,
 'max_cat_to_onehot': None,
 'max_delta_step': None,
 'max_depth': 9,
 'max_leaves': None,
 'min_child_weight': 0.01,
 'missing': nan,
 'monotone_constraints': None,
 'multi_strategy': None,
 'n_estimators': 50,
 'n_jobs': None,
 'num_parallel_tree': None,
 'random_state': None,
 'reg_alpha': 0.05,
 'reg_lambda': None,
 'sampling_method': None,
 'scale_pos_weight': None,
 'subsample': 0.5,
 'tree_method': None,
 'validate_parameters': None,
 'verbosity': None,
 'eta': 0.105}

### LightGBM

In [19]:
from lightgbm import LGBMRegressor
from sklearn.model_selection import ParameterSampler

In [29]:
dict_param = {
    'n_estimator': [10, 20, 50, 100, 200, 500],
    'max_depth': np.asarray([5, 7, 9, 11, 13]).tolist(),
    'num_leaves': ((np.power(2, np.asarray([5, 7, 9, 11, 13])) - 1) * (0.55 + (0.65 - 0.55) * np.random.rand(5))).astype(int).tolist(),
    'min_data_in_leaf': np.linspace(100, 1000, 4).astype(int).tolist(),
    'feature_fraction': np.linspace(0.6, 1, 3),
    'bagging_fraction': np.linspace(0.6, 1, 3),
    'learning_rate': [0.01],
    'verbose': [-1],
}

def custom_sampler(param_grid):
    for params in ParameterSampler(param_grid, n_iter=1e9):
        range_num_leaves = ((0.5 * (2**params['max_depth'] - 1)), (0.7 * (2**params['max_depth']) - 1))
        if(range_num_leaves[0] <= params['num_leaves'] <= range_num_leaves[1]):
            for key, value in params.items():
                params[key] = [value]
            yield params

In [30]:
grid_search = GridSearchCVTrainer(name='LightGBM Regressor', model=LGBMRegressor(), 
                                param_grid=list(custom_sampler(dict_param)), cv = 5, n_jobs=2)
grid_search.fit(X_train, y_train_log)

lgbmr_model = grid_search.best_estimator_
lgbmr_model.fit(X_train, y_train_log)

Training combination 1080/1080


LGBMRegressor(bagging_fraction=0.6, feature_fraction=1.0, learning_rate=0.01,
              max_depth=7, min_data_in_leaf=100, n_estimator=10, num_leaves=81,
              verbose=-1)

In [31]:
evaluate_model(lgbmr_model, 'LightGBM regressor model', X_test, y_test, y_logscale=True)

LightGBM regressor model's evaluation results:
 - Mean squared error:      55.69
 - Root mean squared error: 7.46
 - Mean absolute error:     7.24
 - R2 error:                -13.51
 - F1 score:                0.00
 - Precision:               0.00
 - Recall:                  0.02
 - Accuracy:                0.02
-------------------------------------------------



c:\Users\aupho\AppData\Local\Programs\Python\Python311\Lib\site-packages\lightgbm\basic.py:1218: UserWarning: Converting data to scipy sparse matrix.
  _log_warning("Converting data to scipy sparse matrix.")


In [32]:
lgbmr_model.get_params()

{'boosting_type': 'gbdt',
 'class_weight': None,
 'colsample_bytree': 1.0,
 'importance_type': 'split',
 'learning_rate': 0.01,
 'max_depth': 7,
 'min_child_samples': 20,
 'min_child_weight': 0.001,
 'min_split_gain': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'num_leaves': 81,
 'objective': None,
 'random_state': None,
 'reg_alpha': 0.0,
 'reg_lambda': 0.0,
 'subsample': 1.0,
 'subsample_for_bin': 200000,
 'subsample_freq': 0,
 'verbose': -1,
 'n_estimator': 10,
 'min_data_in_leaf': 100,
 'feature_fraction': 1.0,
 'bagging_fraction': 0.6}

### Stacked model:

In [20]:
from mlxtend.regressor import StackingCVRegressor

Define component models:

In [30]:
elastic_model = ElasticNet(alpha=0.1, l1_ratio=0.0, max_iter=100000, selection='random')

svr_model = SVR(C=1000, gamma=1e-05)

rfr_model = RandomForestRegressor(max_depth=2000, 
                                  max_features=50, 
                                  min_samples_split=25,
                                  min_samples_leaf=1,
                                  n_estimators=1024,
                                  n_jobs=-1)

xgb_model = XGBRegressor(eta=0.105, 
                         gamma=1.0, 
                         max_depth=9, 
                         min_child_weight=0.01, 
                         subsample=0.5, 
                         reg_alpha=0.05, 
                         n_estimators=50)

lgbmr_model = LGBMRegressor(bagging_fraction=0.6, 
                            feature_fraction=1.0, 
                            learning_rate=0.01,
                            max_depth=7, 
                            min_data_in_leaf=100, 
                            n_estimator=10,
                            num_leaves=81,
                            verbose=-1)

Define blended model:

In [31]:
stack_gen = StackingCVRegressor(regressors=(xgb_model, lgbmr_model, svr_model, elastic_model, rfr_model),
                                meta_regressor=rfr_model,
                                use_features_in_secondary=True, n_jobs=-1)
stack_gen.fit(X_train, y_train_log)

c:\Users\aupho\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:658: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 355.71805629734564, tolerance: 0.11916336274031106
  model = cd_fast.sparse_enet_coordinate_descent(


StackingCVRegressor(meta_regressor=RandomForestRegressor(max_depth=2000,
                                                         max_features=50,
                                                         min_samples_split=25,
                                                         n_estimators=1024,
                                                         n_jobs=-1),
                    n_jobs=-1,
                    regressors=(XGBRegressor(base_score=None, booster=None,
                                             callbacks=None,
                                             colsample_bylevel=None,
                                             colsample_bynode=None,
                                             colsample_bytree=None, device=None,
                                             early_stopping_rounds=None,
                                             enable_categorical=False,
                                             et...
                                LGBMRegressor(bagging_fraction=0.6,
                                              feature_fraction=1.0,
                                              learning_rate=0.01, max_depth=7,
                                              min_data_in_leaf=100,
                                              n_estimator=10, num_leaves=81,
                                              verbose=-1),
                                SVR(C=1000, gamma=1e-05),
                                ElasticNet(alpha=0.1, l1_ratio=0.0,
                                           max_iter=100000,
                                           selection='random'),
                                RandomForestRegressor(max_depth=2000,
                                                      max_features=50,
                                                      min_samples_split=25,
                                                      n_estimators=1024,
                                                      n_jobs=-1)),
                    use_features_in_secondary=True)

In [33]:
evaluate_model(stack_gen, 'Stacking model', X_test, y_test, y_logscale=True, save_to_file=True)

c:\Users\aupho\AppData\Local\Programs\Python\Python311\Lib\site-packages\lightgbm\basic.py:1218: UserWarning: Converting data to scipy sparse matrix.
  _log_warning("Converting data to scipy sparse matrix.")


Stacking model's evaluation results:
 - Mean squared error:      47.77
 - Root mean squared error: 6.91
 - Mean absolute error:     6.11
 - R2 error:                -11.45
 - F1 score:                0.00
 - Precision:               0.00
 - Recall:                  0.01
 - Accuracy:                0.01
-------------------------------------------------

